In [2]:
import pandas as pd
import numpy as np

In [3]:
import re
from gensim.models import KeyedVectors
from flashtext import KeywordProcessor
import tqdm

import nltk
from nltk.corpus import brown
from gensim.models import Word2Vec
import gensim.models.keyedvectors as word2vec
import multiprocessing

In [2]:
punc_sign = r"\ə\ᴵ\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&\\…\/\{\}\''\[\]\_\/\@\$\%\^\&\*\(\)\+\#\:\!\-\;\!\"\\(\),\.?'+`~$=|•！？。＂＃＄％＆＇（）＊＋，－／：；<>＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟‧﹏"

puncts = [',', '.', '"', ':', ')', '(', '!', '?', '|', ';', "'", '$', '&',
    '/', '[', ']', '>', '%', '=', '#', '+', '\\', '•',  '~', '@', '£',
    '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',
    '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '“', '★', '”',
    '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾',
    '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼', '⊕', '▼',
    '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
    'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»',
    '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø',
    '¹', '≤', '‡', '√', '«', '»', '´', 'º', '¾', '¡', '§', '£', '₤', 'ə', '√',
    'ᴵ', '∞', 'θ', '÷', 'α', '•', 'à', '−', 'β', '∅', '³', 'π', '‘', '₹', '´', '£', '€',
    '×','™', '√', '²', '—', '…', ':', ';', '•', '！', '?', '$', '＄', '％', '＆', '（', '）', '-', '*']

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 
                'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 
                'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 
                'qoura': 'quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 
                'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'em': 'them',
                'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 
                'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'etherium': 'ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', '2k19':'2019', 'qouta': 'quota', 
                'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 
                'demonitisation': 'demonetization', 'demonitization': 'demonetization', 
                'demonetisation': 'demonetization', 'pokémon': 'pokemon', 'n*gga':'nigga', 'p*':'pussy', 
                'b***h':'bitch', 'a***h****':'asshole', 'a****le-ish':'asshole', 'b*ll-s***':'bullshit', 'd*g':'dog', 
                'st*up*id':'stupid','d***':'dick','di**':'dick',"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", 
                "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", 
                "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
                "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is",
                "they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
                "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will",
                "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have",
                "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would",
                "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling',
                'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora',
                'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best',
                'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data',
                '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization',
                'demonitization': 'demonetization', 'demonetisation': 'demonetization','\u200b': ' ', '\ufeff': '', 'करना': '', 'है': '',
                'sh*tty': 'shitty','s**t':'shit',
                'nigg*r':'nigger','bulls**t':'bullshit','n*****':'nigger',
                'p*ssy':'pussy','p***y':'pussy',
                'f***':'fuck','f*^k':'fuck','f*cked':'fucked','f*ck':'fuck','f***ing':'fucking', 'F*CKING': 'fucking',
                'sh*t':'shit', 'su*k':'suck', 'a**holes':'assholes','a**hole':'asshole',
                'di*k':'dick', 'd*ck': 'dick', 'd**k':'dick', 'd***':'dick',
                'bull**it':'bullshit', 'c**t':'cunt', 'cu*t':'cunt', 'c*nt':'cunt','troʊl':'trool',
                'trumpian':'bombast','realdonaldtrump':'trump','drumpf':'trump','trumpist':'trump',
                "i'ma": "i am","is'nt": "is not","‘I":'I',
                'ᴀɴᴅ':'and','ᴛʜᴇ':'the','ʜᴏᴍᴇ':'home','ᴜᴘ':'up','ʙʏ':'by','ᴀᴛ':'at','…and':'and','civilbeat':'civil beat',\
                'TrumpCare':'Trump care','Trumpcare':'Trump care', 'OBAMAcare':'Obama care','ᴄʜᴇᴄᴋ':'check','ғᴏʀ':'for','ᴛʜɪs':'this','ᴄᴏᴍᴘᴜᴛᴇʀ':'computer',\
                'ᴍᴏɴᴛʜ':'month','ᴡᴏʀᴋɪɴɢ':'working','ᴊᴏʙ':'job','ғʀᴏᴍ':'from','Sᴛᴀʀᴛ':'start','gubmit':'submit','CO₂':'carbon dioxide','ғɪʀsᴛ':'first',\
                'ᴇɴᴅ':'end','ᴄᴀɴ':'can','ʜᴀᴠᴇ':'have','ᴛᴏ':'to','ʟɪɴᴋ':'link','ᴏғ':'of','ʜᴏᴜʀʟʏ':'hourly','ᴡᴇᴇᴋ':'week','ᴇɴᴅ':'end','ᴇxᴛʀᴀ':'extra',\
                'Gʀᴇᴀᴛ':'great','sᴛᴜᴅᴇɴᴛs':'student','sᴛᴀʏ':'stay','ᴍᴏᴍs':'mother','ᴏʀ':'or','ᴀɴʏᴏɴᴇ':'anyone','ɴᴇᴇᴅɪɴɢ':'needing','ᴀɴ':'an','ɪɴᴄᴏᴍᴇ':'income',\
                'ʀᴇʟɪᴀʙʟᴇ':'reliable','ғɪʀsᴛ':'first','ʏᴏᴜʀ':'your','sɪɢɴɪɴɢ':'signing','ʙᴏᴛᴛᴏᴍ':'bottom','ғᴏʟʟᴏᴡɪɴɢ':'following','Mᴀᴋᴇ':'make',\
                'ᴄᴏɴɴᴇᴄᴛɪᴏɴ':'connection','ɪɴᴛᴇʀɴᴇᴛ':'internet','financialpost':'financial post', 'ʜaᴠᴇ':' have ', 'ᴄaɴ':' can ', 'Maᴋᴇ':' make ', 'ʀᴇʟɪaʙʟᴇ':' reliable ', 'ɴᴇᴇᴅ':' need ',
                'ᴏɴʟʏ':' only ', 'ᴇxᴛʀa':' extra ', 'aɴ':' an ', 'aɴʏᴏɴᴇ':' anyone ', 'sᴛaʏ':' stay ', 'Sᴛaʀᴛ':' start', 'SHOPO':'shop','ᴀ':'A',
                'theguardian':'the guardian','deplorables':'deplorable', 'theglobeandmail':'the globe and mail', 'justiciaries': 'justiciary','creditdation': 'Accreditation',
                'doctrne':'doctrine','fentayal': 'fentanyl','designation-': 'designation','CONartist' : 'con-artist','Mutilitated' : 'Mutilated','Obumblers': 'bumblers',
                'negotiatiations': 'negotiations','dood-': 'dood','irakis' : 'iraki','cooerate': 'cooperate','COx':'cox','racistcomments':'racist comments','envirnmetalists': 'environmentalists',
                'SB91':'senate bill','tRump':'trump','utmterm':'utm term','FakeNews':'fake news','Gʀᴇat':'great','ʙᴏᴛtoᴍ':'bottom','washingtontimes':'washington times','garycrum':'gary crum','htmlutmterm':'html utm term',
                'RangerMC':'car','TFWs':'tuition fee waiver','SJWs':'social justice warrior','Koncerned':'concerned','Vinis':'vinys','Yᴏᴜ':'you',
                'trumpists': 'trump', 'trumpkins': 'trump','trumpism': 'trump','trumpsters':'trump','thedonald':'trump',
                'trumpty': 'trump', 'trumpettes': 'trump','trumpland': 'trump','trumpies':'trump','trumpo':'trump',
                'drump': 'trump', 'dtrumpview': 'trump','drumph': 'trump','trumpanzee':'trump','trumpite':'trump',
                'chumpsters': 'trump', 'trumptanic': 'trump', 'itʻs': 'it is', 'donʻt': 'do not','pussyhats':'pussy hats',
                'trumpdon': 'trump', 'trumpisms': 'trump','trumperatti':'trump', 'legalizefreedom': 'legalize freedom',
                'trumpish': 'trump', 'ur': 'you are','twitler':'twitter','trumplethinskin':'trump','trumpnuts':'trump','trumpanzees':'trump',
                'justmaybe':'just maybe','trumpie':'trump','trumpistan':'trump','trumphobic':'trump','piano2':'piano','trumplandia':'trump',
                'globalresearch':'global research','trumptydumpty':'trump','frank1':'frank','trumpski':'trump','trumptards':'trump',
                'alwaysthere':'always there','clickbait':'click bait','antifas':'antifa','dtrump':'trump','trumpflakes':'trump flakes',
                'trumputin':'trump putin','fakesarge':'fake sarge','civilbot':'civil bot','tumpkin':'trump','trumpians':'trump',
                'drumpfs':'trump','dtrumpo':'trump','trumpistas':'trump','trumpity':'trump','trump nut':'trump','tumpkin':'trump',
                'russiagate':'russia gate','trumpsucker':'trump sucker','trumpbart':'trump bart', 'trumplicrat':'trump','dtrump0':'trump',
                'tfixstupid':'stupid','brexit':'<a>','Brexit':'<a>',
               }
               

mispell_dict2 = {'americanophobia': '<q>', 'klastri':'<s>','thisisurl':'<url>','magaphants':'<x>','cheetolini':'<c>','daesh':'<b>',
                'trumpelthinskin':'<n>'}
emoji_re = re.compile(u'['
                        u'\U00010000-\U0010ffff' 
                        u'\U0001F600-\U0001F64F'
                        u'\U0001F300-\U0001F5FF'
                        u'\U0001F30D-\U0001F567'
                        u'\U0001F680-\U0001F6FF'
                        u'\u2122-\u2B55]', re.UNICODE)

kp = KeywordProcessor(case_sensitive=True)
                
mix_mispell_dict = {}
for k, v in mispell_dict.items():
    mix_mispell_dict[k] = v
    mix_mispell_dict[k.lower()] = v.lower()
    mix_mispell_dict[k.upper()] = v.upper()
    mix_mispell_dict[k.capitalize()] = v.capitalize()

for k, v in mix_mispell_dict.items():
    kp.add_keyword(k, v)    
    

kp2 = KeywordProcessor(case_sensitive=True)
for k, v in mispell_dict2.items():
    kp2.add_keyword(k, v)
    

def statistics_upper_words(text):
    upper_count = 0
    for token in text.split():
        if re.search(r'[A-Z]', token):
            upper_count += 1
    return upper_count

def statistics_unique_words(text):
    words_set = set()

    for token in text.split():
        words_set.add(token)

    return len(words_set)

def statistics_characters_nums(text):

    chars_set = set()

    for char in text:
        chars_set.add(char)
    
    return len(chars_set)

def statistics_swear_words(text):
    swear_count = 0
    for swear_word in swear_words:
        if swear_word in text:
            swear_count += 1
    return swear_count

def content_preprocessing(text):
    
    
    # text = text.lower()
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' thisisurl ', text)
    text = emoji_re.sub(' ', text)
    text = kp.replace_keywords(text)
    text = kp2.replace_keywords(text)
    # text = re.sub("[%s]+" %punc_sign , ' ' ,text)
    # emoji_num = len(emoji_re.findall(text))
    text = clean_text(text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    
    # upper_count = statistics_upper_words(text)
    # characters_num = statistics_characters_nums(text)
    # unique_words_num = statistics_unique_words(text)
    # swear_words_num = statistics_swear_words(text)

            
    # return text, swear_words_num, len(text.split()), emoji_num, upper_count, unique_words_num, characters_num
    return text

In [5]:
train_df = pd.read_csv("jigsaw-unintended-bias-in-toxicity-classification/train.csv")
test_df = pd.read_csv("jigsaw-unintended-bias-in-toxicity-classification/test.csv")
train_df['comment_text'] = train_df['comment_text'].fillna('_##_').values
test_df['comment_text'] = test_df['comment_text'].fillna('_##_').values

# shuffling the data
train_df = train_df.sample(frac=1).reset_index(drop=True)

print("Train shape : ", train_df.shape)
print("Test shape : ", test_df.shape)


### preprocessing
# x_train, x_train_swear_words, x_train_token_num, x_train_emoji_num, x_train_upper_count, x_train_unique_words_num, x_train_characters_num = zip(*train_df["comment_text"].apply(lambda x: content_preprocessing(x)))
x_train = train_df["comment_text"].apply(lambda x: content_preprocessing(x))

# x_train_swear_words = np.array(x_train_swear_words, dtype=np.long).reshape(-1, 1)
# x_train_token_num = np.array(x_train_token_num, dtype=np.long).reshape(-1, 1)
# x_train_emoji_num = np.array(x_train_emoji_num, dtype=np.long).reshape(-1, 1)
# x_train_upper_count = np.array(x_train_upper_count, dtype=np.long).reshape(-1, 1)
# x_train_unique_words_num = np.array(x_train_unique_words_num, dtype=np.long).reshape(-1, 1)
# x_train_characters_num = np.array(x_train_characters_num, dtype=np.long).reshape(-1, 1)

y_aux_train = np.array(train_df[['severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']])

# x_test, x_test_swear_words, x_test_token_num, x_test_emoji_num, x_test_upper_count, x_test_unique_words_num, x_test_characters_num  = zip(*test_df["comment_text"].apply(lambda x: content_preprocessing(x)))
x_test = test_df["comment_text"].apply(lambda x: content_preprocessing(x))


Train shape :  (1804874, 45)
Test shape :  (97320, 2)


In [175]:
words_list = [item for sublist in sentences for item in sublist]

In [178]:
len(words_list)

116659279

In [177]:
len(set(words_list))

439856

In [6]:
sentences = pd.DataFrame(x_train).values.tolist() + pd.DataFrame(x_test).values.tolist()

for i, s in enumerate(sentences):
    sentences[i] = s[0].split()

In [16]:
EMB_DIM = 300

w2v_5 = Word2Vec(sentences, size = EMB_DIM, window = 5, min_count = 5, negative = 15, iter = 5, workers = multiprocessing.cpu_count())


In [11]:
word_vec = w2v.wv

In [13]:
word_vec.similar_by_word('Saturday')

/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Monday', 0.8106446862220764),
 ('Friday', 0.7989753484725952),
 ('Sunday', 0.789634108543396),
 ('Tuesday', 0.7655911445617676),
 ('Thursday', 0.7571506500244141),
 ('Sat', 0.7209328413009644),
 ('Wednesday', 0.6740602254867554),
 ('Tues', 0.6642817258834839),
 ('monday', 0.6493574976921082),
 ('afternoon', 0.6327422261238098)]

In [15]:
word_vec.similar_by_word('people')

/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('folks', 0.847642183303833),
 ('individuals', 0.695095419883728),
 ('Americans', 0.6914371252059937),
 ('Canadians', 0.655942976474762),
 ('ppl', 0.6436157822608948),
 ('men', 0.6315521001815796),
 ('citizens', 0.6169369220733643),
 ('persons', 0.6133850812911987),
 ('People', 0.6105722188949585),
 ('folk', 0.6077068448066711)]

In [10]:
w2v.save("word2vec_toxic.model")

In [17]:
word_vec_5 = w2v_5.wv

In [18]:
word_vec_5.similar_by_word('Saturday')

/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Monday', 0.8308067321777344),
 ('Friday', 0.8088524341583252),
 ('Sunday', 0.7910346984863281),
 ('Tuesday', 0.7748814225196838),
 ('Thursday', 0.7702040672302246),
 ('Sat', 0.7296500205993652),
 ('monday', 0.6901320219039917),
 ('Wednesday', 0.6893444061279297),
 ('mornings', 0.6730568408966064),
 ('afternoon', 0.6682068109512329)]

In [19]:
word_vec_5.similar_by_word('people')

/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('folks', 0.8420507907867432),
 ('ppl', 0.6911355257034302),
 ('individuals', 0.6901845932006836),
 ('Americans', 0.6849813461303711),
 ('Canadians', 0.6536723971366882),
 ('men', 0.635567843914032),
 ('persons', 0.614833414554596),
 ('folk', 0.6124849319458008),
 ('citizens', 0.6069498062133789),
 ('People', 0.6041996479034424)]

In [4]:
# load w2v pretrained model
mod = Word2Vec.load("word2vec_5_toxic.model")

In [5]:
mod.wv.word_vec('a')

array([ 1.1354041e+00,  2.0508060e+00, -1.5045446e+00, -1.8079064e+00,
       -7.2623053e+00,  3.2742286e+00, -1.7727764e+00,  1.6742339e+00,
       -2.1809361e+00, -1.4824820e+00, -1.6520038e-01, -8.7806219e-01,
        2.1176753e+00, -1.9665916e+00,  4.0627003e+00,  1.2337866e+00,
       -5.0124055e-01, -1.4412597e+00, -4.0134898e-01,  5.8231723e-01,
       -5.8083099e-01,  1.4449546e+00, -3.1821087e-01, -5.9059674e-01,
       -1.0887691e+00,  1.0027692e+00, -1.3858904e+00, -1.6214668e+00,
       -3.6171106e-01, -2.1032393e+00, -2.6351700e+00, -8.1643164e-01,
        7.2800565e-01, -2.3075898e-01, -2.5022948e+00,  3.3966476e-01,
        3.2912934e+00, -9.7948372e-01, -4.3140543e-01, -6.7572999e-01,
        1.4833511e+00, -1.3219533e+00, -1.0327908e+00,  1.0059941e+00,
        1.8049970e+00,  1.4624935e-01, -6.9917518e-01, -1.3378119e+00,
        3.5584950e+00,  8.9679855e-01,  1.7294911e+00,  7.5850630e-01,
        4.8537707e+00, -6.1280439e-03, -2.1086326e+00, -1.1157238e+00,
      